In [ ]:
# PyTorchが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env MKL_NUM_THREADS=1

from torch import set_num_threads, set_num_interop_threads
num_threads = 1
set_num_threads(num_threads)
set_num_interop_threads(num_threads)

In [ ]:
#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchinfo
from tqdm import tqdm
import utils

## ミニバッチ法のPyTorchによる実装
PyTorchでミニバッチ法を実装してみましょう。
基礎編で使った2次元データとMLPモデルを例に実験します。

まずは一度に全てのデータを使ってロスの計算を行うバッチ学習を試してみます。

In [ ]:
import time

# データ点の取得 (データセットのサイズは1100です。)
x, t = utils.dataset_for_mlp()

# numpy -> torch.tensor
x = torch.from_numpy(x).float()
t = torch.from_numpy(t).float()

# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=2, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=1),
    nn.Sigmoid(),
)

# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# トレーニング
for i_epoch in range(1):
    # 時間の測定
    start_time = time.time()

    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x)

    # ロスの計算
    loss = loss_fn(y_pred, t)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    print(f'epoch = {i_epoch}, time = {time.time() - start_time: .3f} sec, loss = {loss}')

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# パーセプトロンの出力を等高線プロット
utils.plot_prediction(model, ax=ax)

# データ点をプロット
utils.plot_datapoint(x, t, ax=ax)

# 図を表示
plt.show()

次に1行ずつロスを計算するオンライン学習で学習を実行します。

In [ ]:
# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=2, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=1),
    nn.Sigmoid(),
)

# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# トレーニング
for i_epoch in range(1):
    # 時間の測定
    start_time = time.time()

    # 1行ずつロスを計算し、重みを更新する。
    for xb, tb in zip(x, t):
        # モデルをトレーニングモードにする。
        model.train()

        # 順伝搬
        y_pred = model(xb)

        # ロスの計算
        loss = loss_fn(y_pred, tb)

        # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
        # これをしないと、勾配の値はそれまでの値との和がとられる。
        optimizer.zero_grad()

        # 誤差逆伝播。各パラメータの勾配が計算される。
        loss.backward()

        # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
        optimizer.step()

    print(f"epoch = {i_epoch}, time = {time.time() - start_time: .3f} sec, loss = {loss}")

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# パーセプトロンの出力を等高線プロット
utils.plot_prediction(model, ax=ax)

# データ点をプロット
utils.plot_datapoint(x, t, ax=ax)

# 図を表示
plt.show()

- オンライン学習では1ステップあたりの時間は短いですが、1100回重みの更新をする必要があるためトータルの時間はバッチ法よりも長くかかっています。
- 今回は全データを１回ずつ使用して(エポック数1で)学習したためバッチ法では学習が十分に進んでいません。エポック数を増やすことで学習をさらに進めることができます。

次にミニバッチ法を試してみましょう。バッチサイズは10としてみます。

In [ ]:
# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=2, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=1),
    nn.Sigmoid(),
)

# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# トレーニング
for i_epoch in range(1):
    # 時間の測定
    start_time = time.time()

    # 10行ずつロスを計算し、重みを更新する。
    for i_batch in range(110):
        # 10行ずつデータを切り出す
        xb = x[10 * i_batch : 10 * (i_batch + 1)]
        tb = t[10 * i_batch : 10 * (i_batch + 1)]

        # モデルをトレーニングモードにする。
        model.train()

        # 順伝搬
        y_pred = model(xb)

        # ロスの計算
        loss = loss_fn(y_pred, tb)

        # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
        # これをしないと、勾配の値はそれまでの値との和がとられる。
        optimizer.zero_grad()

        # 誤差逆伝播。各パラメータの勾配が計算される。
        loss.backward()

        # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
        optimizer.step()

    print(f"epoch = {i_epoch}, time = {time.time() - start_time: .3f} sec, loss = {loss}")

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# パーセプトロンの出力を等高線プロット
utils.plot_prediction(model, ax=ax)

# データ点をプロット
utils.plot_datapoint(x, t, ax=ax)

# 図を表示
plt.show()

1回の重み更新で10イベントを処理するので、全体として110ステップ分処理することになりました。
振る舞いはバッチ学習とオンライン学習の中間くらいになっています。

今回はデータセットがシンプルだったため、オンライン学習でも問題なく学習が進みましたが、オンライン学習は学習が不安定になることが多いです。
そのため実際の深層学習モデル学習の際は、バッチサイズが32 ~ 2048程度のミニバッチ学習を使うことが多いです。

ミニバッチの処理をする際、毎度データをバッチに切り出すコードを書くのは大変なので、通常はDataLoaderというクラスを用いてこれを処理します。
上記のミニバッチのコードは以下のように書き換えることができます。

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

ds = TensorDataset(x, t)
dataloader = DataLoader(ds, batch_size=10)  # バッチサイズとして10を指定します。

# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=2, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=1),
    nn.Sigmoid(),
)

# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# トレーニング
for i_epoch in range(1):
    # 時間の測定
    start_time = time.time()

    # 10行ずつロスを計算し、重みを更新する。
    for xb, tb in dataloader:
        # モデルをトレーニングモードにする。
        model.train()

        # 順伝搬
        y_pred = model(xb)

        # ロスの計算
        loss = loss_fn(y_pred, tb)

        # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
        # これをしないと、勾配の値はそれまでの値との和がとられる。
        optimizer.zero_grad()

        # 誤差逆伝播。各パラメータの勾配が計算される。
        loss.backward()

        # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
        optimizer.step()

    print(f"epoch = {i_epoch}, time = {time.time() - start_time: .3f} sec, loss = {loss}")

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# パーセプトロンの出力を等高線プロット
utils.plot_prediction(model, ax=ax)

# データ点をプロット
utils.plot_datapoint(x, t, ax=ax)

# 図を表示
plt.show()